<a href="https://colab.research.google.com/github/csml1020gk/csml1030-capstone-project/blob/main/spacy_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Working with a text  dataset from one PDF

Extract terms rm from the text and save it's span and label as a line in the training data file

We can then try to scale it up to work with all pdfs
   
At the end, we will develop a baseline model to predict the entity on  a test PDF 



In [1]:
#installations
!pip install spacy


In [2]:
#download language models 
!python -m spacy download en_core_web_sm
import spacy
nlp = spacy.load("en_core_web_sm")
#working with a larger model can be slower but can impact accuracy 

     |████████████████████████████████| 12.0 MB 4.5 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [3]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
#imports
import os 
import pandas as pd
from pandas import DataFrame
import json
from spacy.matcher import Matcher, PhraseMatcher
from spacy.lang.en import English
import re

In [5]:
#functions
def listToString(s): 
    # initialize an empty string
    str1 = "" 
    # traverse in the string  
    for ele in s: 
      str1 += ele
    # return string  
    return str1 



In [6]:
import os
base_path = '/content/drive/MyDrive/week5/files'
save_path = '/content/drive/MyDrive/week5/files'
os.chdir(base_path)
os.getcwd()

'/content/drive/MyDrive/week5/files'

In [7]:
os.listdir()
file_text = open(os.path.join(base_path, 'extracted.txt'),"r")
file_list = file_text.readlines()
file_list

["O2COPL)\\pRoTSSORYOTEDue: On Demand Dated: JulY Ist, L975$63,364.25 R. R. #3Woodstock, OntarioON DEMAND, I promise to pay CHRISTOPHER ECONOMOPOULOS----the sum of SIXtY-THREE THOUSAND, THREE HUNDRED AND SIXTY-FOUR---- ($o:,364.25)--- 25/100 DoLLARStogether with interest at the rate of nil per cent perannum. Value receive'd.GRECO-IBERIAN CERAMICS LIMITED(r*,U3Per:ct,ll\n",
 '!=om(nG)o@\n',
 'oo-to\n',
 '\n',
 'For Minartry Ur. OntyA t\'usago cxcluet du miniltcre@):H:H::"".F.:i*ij,i;ilTi".,",",irectorBusiness CorPorationsAct /LBi :"r"J\',\'ff ""ti6t6 s p a\' acrionsOnlrrlo Corporutlon Numb•rNum6ro do lr comp6gnio on Ontarlo307421TRANScoo•18Ministero dos Solvicesaux consommateuts?#rd;Lr\'\'+h$"Tno"u\'u"are ellective onJANUAFYz a JAIWIEH, 20tzARTICLES OF AMENDMENTSTATUTS DE MODIFICATIONForm 3BusinessCorporalion3ActFormulenum6ro 3Loisur le3compagni6s1. The present name of the corporation is:GRE Cr 0-rIBE2. The name of the corporation is changed to (ifapplicable):D6nomination sociale actuel

In [8]:
#convert list to dataframe 
import pandas as pd 
df_extracted_pages = pd.DataFrame(file_list, columns =['text'])

In [9]:
df_extracted_pages


,text
0,O2COPL)\pRoTSSORYOTEDue: On Demand Dated: JulY...
1,!=om(nG)o@\n
2,oo-to\n
3,\n
4,For Minartry Ur. OntyA t'usago cxcluet du mini...
...,...
202,tq86il Ia_qtiln9^R:s5F:PXiAcxr=iH:ig=tFSrioafi...
203,UJfEo-ozb(E0)()q)/):o-0)ci5oC.{ooo(U-cU)coEEoo...
204,"BqSEd bab:bAi+B""iBqSxtrIS^EH]iO :!c=zaHPrio!+,..."
205,"trJlJstrTLoz$q)\.9E-opoCoctoLo.oLoo(uEooooE(t,..."


In [10]:
#create a processed Doc object for the document.
#input is the text file 
file_name = os.path.join(base_path, "extracted.txt")
file_text = open(file_name).read()
file_doc = nlp(file_text)
#the text file is converted into a processed doc file 

In [11]:
words = []
labels = []

for token in file_doc:
	words.append(token.text)
	labels.append('O') # As most of token will be non-entity (OUT). Replace this later with actual entity a/c the scheme.

df = pd.DataFrame({'word': words, 'label': labels})
df.to_csv('/content/drive/MyDrive/week5/files/ner-token-per-line.txt', index=False) # biluo in extension to indicate the type of encoding, it is ok to keep csv


In [12]:
def show_ents(doc):
  ent_list = [] 
  if doc.ents:
    for ent in doc.ents:
      ent_list.append([ent.text, ent.start_char, ent.end_char, ent.label_])
    return ent_list
  else:
    print('No entities found')


ent_list = show_ents(file_doc)


# Create the pandas DataFrame
df = pd.DataFrame(ent_list, columns = ['Text', 'Start', 'End', 'Label'])

In [13]:
df.sample(5)

,Text,Start,End,Label
948,PresidentDena,101814,101827,PERSON
1726,prosentod,165881,165890,ORG
550,ANNUAL RESOLUTIONS OF,56193,56214,ORG
799,Section 148 of the Business Corporations Act,84257,84301,LAW
275,June 19 75,31169,31179,DATE


In [14]:
data = {'Name':['Current Entity Name', 'Names Forming Amalgamation'],
        'Category':['BasicEntity Data','Basic Entity Data']}

In [15]:
import pandas as pd 
df_output = pd.DataFrame(data)

In [16]:
nlp.pipe_names

['tagger', 'parser', 'ner']

In [17]:
#display default entities 
for ent in file_doc.ents:
  print(ent.text,ent.label_)

JulY Ist PERSON
L975$63,364.25 R. R. #3Woodstock PERSON
CHRISTOPHER DATE
SIXtY-THREE THOUSAND WORK_OF_ART
THREE HUNDRED QUANTITY
Minartry Ur PERSON
t'usago PERSON
ti6t6 s p a' WORK_OF_ART
Corporutlon Numb•rNum6ro PERSON
comp6gnio ORG
Ontarlo307421TRANScoo•18Ministero NORP
onJANUAFYz PRODUCT
20tzARTICLES CARDINAL
0-rIBE2 CARDINAL
Dateof PERSON
Les ORG
Corporation ORG
one CARDINAL
five CARDINAL
5 CARDINAL
Corporation ORG
the City of London GPE
Middlesex GPE
at187 Exeter Road FAC
London GPE
Ontario GPE
Corporation ORG
Corporation ORG
deleted.4.4OYE E PRODUCT
3 CARDINAL
19 CARDINAL
168 CARDINAL
ofthe Business Corporations Act LAW
autoris6e CARDINAL
Al'article 168 PRODUCT
s'il GPE
mois PERSON
ann6s)These NORP
sign6s CARDINAL
GRECO-TBERIAN CERAMICS I LAW
TMTTEI)(Nrma PERSON
Zouheirb PERSON
IICS LIMITEDwas PERSON
the Business ORG
June 3 DATE
FILEDJrlil PERSON
1975MINISTRY CARDINAL
Corporation ORG
655 CARDINAL
CanterburyStreet ORG
3lrloodstock MONEY
Ontario GPE
OBJECTS ORG
3z CARDINAL
three th

In [18]:
#load existing pretrained spacy ner model with a built in ner component
#then get the NER using get_pipe()



In [19]:
#tokenization - identify basic units 
for token in file_doc:
  print (token, token.idx)
#preserves the starting index of the tokens

Streaming output truncated to the last 5000 lines.
CERAMICS 141232
LIMITED(the 141241
" 141253
Corporation")I. 141254
APPROVAL 141270
OFANCIAL 141279
STATEMENTSBE 141288
TT 141301
RESOLVED 141304
THAT 141313
: 141317
the 141318
financial 141322
statements 141332
of 141343
the 141346
Corporation 141350
for 141362
the 141366
fiscal 141370
period 141377
ended 141384
March 141390
31,2018 141396
, 141403
whichhave 141405
been 141415
prepared 141420
by 141429
the 141432
accountant 141436
of 141447
the 141450
Corporation 141454
be 141466
and 141469
the 141473
same 141477
are 141482
hereby 141486
approved 141493
, 141501
and 141502
the 141506
sole 141510
director 141515
is 141524
hereby 141527
authorized 141534
to 141545
sign 141548
the 141553
Balance 141557
Sheet 141565
to 141571
evidence 141574
such 141583
approval 141588
, 141596
and 141597
be 141601
it 141604
further 141607
resolved 141615
that 141624
the 141629
said 141633
financial 141638
statements 141648
be 141659
submitted 141662
to 1

In [20]:
#attributes of each token
for token in file_doc:
  print (token, token.idx, token.text_with_ws,
    token.is_alpha, token.is_punct, token.is_space,
      token.shape_, token.is_stop)
#the tokenizer can be customized to our use case 

Streaming output truncated to the last 5000 lines.
be 141798 be  True False False xx True
and 141801 and  True False False xxx True
are 141805 are  True False False xxx True
hereby 141809 hereby  True False False xxxx True
requested 141816 requested  True False False xxxx False
to 141826 to  True False False xx True
transact 141829 transact  True False False xxxx False
the 141838 the  True False False xxx True
annual 141842 annual  True False False xxxx False
business 141849 business  True False False xxxx False
ofthe 141858 ofthe  True False False xxxx False
Corporation 141864 Corporation True False False Xxxxx False
. 141875 . False True False . False
THE 141876 THE  True False False XXX True
FOREGOING 141880 FOREGOING  True False False XXXX False
RESOLUTION 141890 RESOLUTION  True False False XXXX False
is 141901 is  True False False xx True
hereby 141904 hereby  True False False xxxx True
consented 141911 consented  True False False xxxx False
to 141921 to  True False False xx True

In [21]:
#statistical analysis 
from collections import Counter
words = [token.text for token in file_doc
  if not token.is_stop and not token.is_punct]
word_freq = Counter(words)
common_words = word_freq.most_common(5)
print (common_words)
unique_words = [word for (word, freq) in word_freq.items() if freq == 1]
print (unique_words)

[('Corporation', 250), ('\n', 193), ('directors', 149), ('Business', 133), ('Act', 119)]
['O2COPL)\\pRoTSSORYOTEDue', 'Demand', 'JulY', 'Ist', 'L975$63,364.25', '3Woodstock', 'OntarioON', 'DEMAND', 'promise', 'ECONOMOPOULOS----the', 'THOUSAND', 'FOUR----', 'o:,364.25)---', '25/100', 'DoLLARStogether', 'perannum', 'Value', "receive'd", 'LIMITED(r*,U3Per', 'om(nG)o@', 'Minartry', "t'usago", 'cxcluet', 'miniltcre@):H', 'H::""', 'F.:i*ij', 'i;ilTi"', ',",",irectorBusiness', 'CorPorationsAct', '/LBi', 'r"J\',\'ff', 'ti6t6', 'acrionsOnlrrlo', 'Corporutlon', 'Numb•rNum6ro', 'comp6gnio', 'Ontarlo307421TRANScoo•18Ministero', 'Solvicesaux', 'consommateuts?#rd;Lr\'\'+h$"Tno"u\'u"are', 'ellective', 'onJANUAFYz', 'JAIWIEH', '20tzARTICLES', 'AMENDMENTSTATUTS', 'DE', 'MODIFICATIONForm', '3BusinessCorporalion3ActFormulenum6ro', '3Loisur', 'le3compagni6s1', 'GRE', 'Cr', '0-rIBE2', 'ifapplicable):D6nomination', 'sociale', 'actuelle', 'Nouvelle', 'd6nomination', 'soclale', 'il', 'alieu):3', 'Dateof', 'am

In [22]:
#create the training data to teach the named entity model
#TEXTS is a list of sentences in the page 
#find all mentions of WELL CANADA LTD. so we can create training data to teach a model to recognize them as COM 
#write a pattern which matches  WELL CANADA LTD. 






#file_name = os.path.join(save_path, "small_data.txt")
#TEXTS = open(file_name).read()
#matcher = PhraseMatcher(nlp.vocab, attr = "LOWER")

# Token which matches WELL CANADA LTD.
#pattern = [{"TEXT": "WELL CANADA LTD."}]

# Add patterns to the matcher and check the result
#matcher.add("COM", None, pattern)
#for doc in nlp.pipe(TEXTS):
#    print([doc[start:end] for match_id, start, end in matcher(doc)])





import spacy
from spacy.matcher import PhraseMatcher
#corpus = df_extracted_pages['text'][2]#string 
nlp = spacy.load("en_core_web_sm")
#doc = nlp(corpus)
matcher = PhraseMatcher(nlp.vocab)
terms = ["SHAREHOLDER"]#terms to match 

train_data = open(os.path.join(save_path, 'saved_ent.csv'), 'w')
# Only run nlp.make_doc to speed things up
patterns = [nlp.make_doc(text) for text in terms]
matcher.add("TerminologyList", patterns)#add the pattern to the matcher object 

matches = matcher(file_doc)
annotations = []#saved list of annotations, one in this case 
list_str = "("
for match_id, start, end in matches:
    span = file_doc[start:end]
    annotations.append((span.text, {'entities': [(start, end, 'COM')]}))
    
for i in annotations:
    train_data.write(str(i)+'\n')
train_data.close()



print(annotations)
#training data is in spacy format 

[('SHAREHOLDER', {'entities': [(5658, 5659, 'COM')]}), ('SHAREHOLDER', {'entities': [(8021, 8022, 'COM')]}), ('SHAREHOLDER', {'entities': [(9081, 9082, 'COM')]}), ('SHAREHOLDER', {'entities': [(16066, 16067, 'COM')]})]


In [23]:
print(annotations)
#WELL CANADA appears only once in this document , at the beginning 

[('SHAREHOLDER', {'entities': [(5658, 5659, 'COM')]}), ('SHAREHOLDER', {'entities': [(8021, 8022, 'COM')]}), ('SHAREHOLDER', {'entities': [(9081, 9082, 'COM')]}), ('SHAREHOLDER', {'entities': [(16066, 16067, 'COM')]})]


In [24]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm

In [25]:
#create your custom entity data for the input text where the named entity is to be identified by the model during the testing period

TRAIN_DATA = annotations

In [26]:
#define the variables required for the training model
model = None
output_dir=Path(save_path)
n_iter=10
#to save time,  only tried ten iterations 

In [27]:
#load a blank model for the process to carry out the NER action and set up the pipeline with only NER using create_pipe function
#load the model

if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

#set up the pipeline

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe('ner')

#the pipleline can be expanded later to include other steps 
    

Created blank 'en' model


In [28]:
for _, ann in TRAIN_DATA:
    for ent in ann.get('entities'):# we  only have one entity 
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            nlp.update(
                [text],  
                [annotations],  
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)
    #in ten interations, the loss comes down to 68%
    


100%|██████████| 4/4 [00:00<00:00, 26.09it/s]


{'ner': 3.2147570848464966}


100%|██████████| 4/4 [00:00<00:00, 35.65it/s]


{'ner': 2.756849467754364}


100%|██████████| 4/4 [00:00<00:00, 37.10it/s]


...

{'ner': 8.658655684845584e-05}


100%|██████████| 4/4 [00:00<00:00, 32.51it/s]


{'ner': 0.0058392719229303225}


100%|██████████| 4/4 [00:00<00:00, 37.95it/s]

{'ner': 1.3478623719191674e-05}


In [29]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(os.path.join(output_dir, 'nlp_model'))
    print("Saved model to", output_dir)

#save the model 

Saved model to /content/drive/MyDrive/week5/files


In [30]:

try_model = spacy.load(os.path.join(output_dir, 'nlp_model'))#load the  trained model 

In [31]:
try_model.entity.labels#our model can only predict COM labels at the moment 

('COM',)

In [32]:
#input text file as string 
test_file = open(os.path.join(base_path, "test_data.txt"), "r")
file_str = listToString(test_file.readlines())

test_doc = try_model(file_str)

#print(test_doc)
for ent in test_doc.ents:
  print(f'{ent.label_:{20}} - {ent.text}')
#the ner model could not find any entities becuse of the large losses. we have very small training data too 

# The training data must have many examples in order to generate a good ner model that can predict entities from test data. 